<a href="https://colab.research.google.com/github/imustakimk/NLP-Project/blob/main/Multimodal_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 7.0 MB/s 
     |████████████████████████████████| 6.8 MB 50.1 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 895 kB 60.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoFeatureExtractor, AutoTokenizer, TFVisionEncoderDecoderModel
from PIL import Image
import requests

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
decoder_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# initialize a bert2gpt2 from a pretrained BERT and GPT2 models. Note that the cross-attention layers will be randomly initialized
model = TFVisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    "google/vit-base-patch16-224-in21k", "gpt2"
)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")

        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="tf").pixel_values
    output_ids = model.generate(pixel_values,decoder_start_token_id=model.config.decoder.bos_token_id, **gen_kwargs)
    preds = decoder_tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

predict_step(['/content/1000344755_img.jpg'])


All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

Some layers of TFGPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['tf_vision_encoder_decoder_model/transformer/h_._0/crossattention/c_proj/weight:0', 'tf_vision_encoder_decoder_model/transformer/h_._2/crossattention/q_attn/bias:0', 'tf_vision_encoder_decoder_model/transformer/h_._1/crossattention/q_attn/weight:0', 'tf_vision_encoder_decoder_model/transformer/h_._0/ln_cross_attn/beta:0', 'tf_vision_encoder_decoder_model/transformer/h_._6/crossattention/c_proj/weight:0', 'tf_vision_encoder_decoder_model/transformer/h_._8/crossattention/c_

['"I don\'t know if I\'m going to be able to do it']

In [ ]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
    images = []
    for image_path in image_paths:
        i_image = Image.open(image_path)
        if i_image.mode != "RGB":
            i_image = i_image.convert(mode="RGB")

        images.append(i_image)

    pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
    output_ids = model.generate(pixel_values,decoder_start_token_id=model.config.decoder.bos_token_id, **gen_kwargs)
    preds = decoder_tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]
    return preds

predict_step(['/content/1000344755_img.jpg'])


Downloading:   0%|          | 0.00/4.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/937M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

['a man is standing outside of a building with a window']

In [ ]:
#import gradio as gr
from transformers import AutoTokenizer,AutoModelForQuestionAnswering
import torch

def inference(question,context):
    question_first=bool(tokenizer.padding_side=='right')
    max_answer_len=5
    encoded_text=tokenizer.encode_plus(question,context,padding='longest',
                    truncation="longest_first" ,
                    max_length=512,
                    stride=30,
                    return_tensors="pt",
                    return_token_type_ids=False,
                    return_overflowing_tokens=False,
                    return_offsets_mapping=False,
                    return_special_tokens_mask=False)
    print('encoded text',encoded_text)
    input_ids=encoded_text['input_ids'].tolist()[0]
    print('Input ids',input_ids)
    # tokenization is completed 

    #input_ids=tokenizer.convert_tokens_to_ids(tokens)
    
    with torch.no_grad():
        outputs=model(**encoded_text)
#     answer_st=outputs.start_logits
#     answer_et=outputs.end_logits
    start_,end_=outputs[:2]
    # this line gives you the starting idx
    answer_start=torch.argmax(start_)
    #this line gives you the ending idx -> +1???
    answer_end=torch.argmax(end_)+1
    #below line ->detokenization
    answer=tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer
    



model=AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased-distilled-squad')
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-uncased-distilled-squad')

'''
nlp_QA=pipeline('question-answering',model=model,tokenizer=tokenizer)
QA_inp={
    'question': 'How many parameters does Bert large have?',
    'context': 'Bert large is really big... it has 24 layers, for a total of 340M parameters.Altogether it is 1.34 GB so expect it to take a couple minutes to download to your Colab instance.'
}

result=nlp_QA(QA_inp)
'''
question='How many parameters does Bert large have?'
context='Bert large is really big... it has 24 layers, for a total of 340M parameters.Altogether it is 1.34 GB so expect it to take a couple minutes to download to your Colab instance.'
print(inference(question,context))

title = 'Question Answering demo with Albert QA transformer and gradio'
gr.Interface(inference,inputs=[gr.inputs.Textbox(lines=7, default=context, label="Context"), gr.inputs.Textbox(lines=2, default=question, label="Question")],
#              outputs=[gr.outputs.Textbox(type="auto",label="Answer")],title = title,theme = "peach").launch()

Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

encoded text {'input_ids': tensor([[  101,  2129,  2116, 11709,  2515, 14324,  2312,  2031,  1029,   102,
         14324,  2312,  2003,  2428,  2502,  1012,  1012,  1012,  2009,  2038,
          2484,  9014,  1010,  2005,  1037,  2561,  1997, 16029,  2213, 11709,
          1012, 10462,  2009,  2003,  1015,  1012,  4090, 16351,  2061,  5987,
          2009,  2000,  2202,  1037,  3232,  2781,  2000,  8816,  2000,  2115,
         15270,  2497,  6013,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}
Input ids [101, 2129, 2116, 11709, 2515, 14324, 2312, 2031, 1029, 102, 14324, 2312, 2003, 2428, 2502, 1012, 1012, 1012, 2009, 2038, 2484, 9014, 1010, 2005, 1037, 2561, 1997, 16029, 2213, 11709, 1012, 10462, 2009, 2003, 1015, 1012, 4090, 16351, 2061, 5987, 2009, 2000, 2202, 1037, 3232, 2781, 2000, 8816, 2000, 2115, 15